<a href="https://colab.research.google.com/github/eringgao/lucy_listings/blob/main/lucyslistings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installations and Declarations

In [ ]:
import pandas as pd
import numpy as np
%pip install geopy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# TYPE_CMU = 0 
# TYPE_ZILLOW = 1 
# TYPE_MANUAL = 2

# Coding Starts Here

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/rapid_data.csv")

print(df)

    bathrooms  bedrooms           city country currency  daysOnZillow  \
0         2.0         4     Pittsburgh     USA      USD             0   
1         1.0         2  Mt Washington     USA      USD             0   
2         1.5         3     Pittsburgh     USA      USD             0   
3         2.0         4     Pittsburgh     USA      USD             0   
4         1.0         1     Pittsburgh     USA      USD             0   
5         1.0         3     Pittsburgh     USA      USD             0   
6         1.0         2     Pittsburgh     USA      USD             0   
7         3.0         5     Pittsburgh     USA      USD             0   
8         1.5         2     Pittsburgh     USA      USD             0   
9         2.0         3     Pittsburgh     USA      USD             0   
10        2.0         3     Pittsburgh     USA      USD             0   
11        1.0         2     Pittsburgh     USA      USD             0   
12        1.0         4     Pittsburgh     USA     

In [ ]:
'''
  Create an OOP structure for how we want to store all of our data.
  Example format: 
'''
class Listing: # abstraction?
  def get_type(self) -> str: 
    # Manual, Zillow, CMU
    pass 
  def get_housing_info(self) -> str: 
    # String data (paragraphs?) about housing
    pass
  def get_address(self) -> dict: 
    pass
  def distance_from_cmu(self) -> float: 
    pass

class SchoolListing(Listing): 
  def __init__(self, address): 
    # Some stuff here
    self.get_type = "CMU"
    # self.distance_from_cmu 


In [ ]:
geolocator = Nominatim()
addresses = df.loc[:, "streetAddress"]
cmu_address = "5000 Forbes Ave, Pittsburgh, PA 15213"
print(addresses)
distances = dict()
for i in range(len(addresses)):
  

0                 2353 Sidney St #2
1                441 Edgemont St #2
2                     200 S 15th St
3                   5317 Forbes Ave
4                4711 Maripoe St #1
5                     1625 Hybla St
6               3202 Hazelhurst Ave
7                  4245 Parkman Ave
8                1007 Galveston Ave
9                   11370 Althea Rd
10                333 Lehigh Ave #2
11    1465 Old Steubenville Pike #B
12                  223 Carnegie Pl
13           5541 Beacon St FLOOR 1
14                 6116 Stanton Ave
15                 2801 Larkins Way
16                   637 Danbury St
17              1806 Rhine St #1804
18              4008 Woolslayer Way
19              3220 1/2 Hardie Way
20              514 Parallel Ave #2
21              2525 Sarah St APT 6
22        230 Clairmont Ave FLOOR 2
23            7708 Westmoreland Ave
24             2507 Wedgemere St #2
25                 252 Atwood St #1
26                    1315 Diana St
Name: streetAddress, dtype: 

/usr/local/lib/python3.8/dist-packages/geopy/geocoders/osm.py:132: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  warnings.warn(
